In [17]:
import logging
import scraper
from sinks.postgres import PostgresDataStorese
import os 
from dotenv import load_dotenv

load_dotenv()

URL = os.getenv("PROXIES_URL")
MAX_WORKERS = int(os.getenv("MAX_WORKERS", 3))
MAX_RETRIES = int(os.getenv("MAX_RETRIES", 5))
DATABASE_URL = os.getenv("DATABASE_URL")
TABLE_NAME = os.getenv("TABLE_NAME")
QUERY_PATH = '../db/releases.sql'
BATCH_SIZE = 100

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def write_to_postgres(p, releases):
    if not releases:
        logging.warning("No releases to write to Postgres.")
        return

    insert_release_sellers(p, releases)
    insert_release_details(p, releases)
    insert_release_wants_haves(p, releases, 'want')
    insert_release_wants_haves(p, releases, 'have')

def insert_release_sellers(p, releases):
    """Insert release sellers data into the release_sellers table."""
    sellers_data = [
        (
            release['release_id'],
            seller.get('image_url'),
            seller.get('rating'),
            seller.get('have'),
            seller.get('want'),
            seller.get('title'),
            seller.get('label'),
            seller.get('catno'),
            seller.get('media_condition'),
            seller.get('media_condition_description'),
            seller.get('seller'),
            seller.get('seller_rating'),
            seller.get('ships_from'),
            seller.get('currency'),
            seller.get('price')
        )
        for release in releases for seller in release.get('sellers', [])
    ]
    query = """
    INSERT INTO release_sellers (
        release_id, image_url, rating, have, want, title, label, catno, media_condition, 
        media_condition_description, seller, seller_rating, ships_from, currency, price
    ) VALUES %s
    """
    p.bulk_insert(query, sellers_data)

def insert_release_details(p, releases):
    """Insert release details data into the release_details table."""
    details_data = [
        (
            release['release_id'],
            release.get('release', {}).get('Have'),
            release.get('release', {}).get('Want'),
            release.get('release', {}).get('Avg Rating'),
            release.get('release', {}).get('Ratings'),
            release.get('release', {}).get('Last Sold'),
            release.get('release', {}).get('Low'),
            release.get('release', {}).get('Median'),
            release.get('release', {}).get('High')
        )
        for release in releases
    ]

    query = """
    INSERT INTO release_details (
        release_id, have, want, avg_rating, ratings, last_sold, low, median, high
    ) VALUES %s
    """
    p.bulk_insert(query, details_data)

def insert_release_wants_haves(p, releases, type_):
    """Insert release wants/haves data into the release_wants or release_haves table."""
    table_name = f"release_{type_}"
    data = [
        (release['release_id'], user)
        for release in releases for user in release.get('stats', {}).get(type_, [])
    ]
    query = f"INSERT INTO {table_name} (release_id, username) VALUES %s"
    p.bulk_insert(query, data)
   
def main():
    p = PostgresDataStore(DATABASE_URL, TABLE_NAME)
    release_ids = p.fetch_ids_from_file(QUERY_PATH)
    
    for i in range(0, len(release_ids), BATCH_SIZE):
        batch_ids = release_ids[i:i + BATCH_SIZE]
        try:
            releases = scraper.run_scraper(URL, batch_ids, MAX_WORKERS)
            write_to_postgres(p, releases)
        except Exception as e:
            logging.error(f"Error processing batch {i//BATCH_SIZE}: {e}")

In [18]:
p = PostgresDataStore(DATABASE_URL, TABLE_NAME)
release_ids = p.fetch_ids_from_file(QUERY_PATH)

for i in range(0, len(release_ids), BATCH_SIZE):
    batch_ids = release_ids[i:i + BATCH_SIZE]
    releases = scraper.run_scraper(URL, batch_ids, MAX_WORKERS)
    write_to_postgres(p, releases)

2024-02-14 16:38:58,896 - INFO - Successfully fetched data: {'release_id': 17, 'release': {'Have': 271, 'Want': 497, 'Avg Rating': 4.38, 'Ratings': 55, 'Last Sold': datetime.date(2023, 12, 17), 'Low': 6.44, 'Median': 16.01, 'High': 21.51}, 'stats': {'have': ['andreeeeaaaaas', 'andygugs', 'asaisthorpe', 'Awesomer', 'bat_howell', 'bjoernmuenchau', 'blancodisco', 'bluediet', 'bobbsq', 'Bonna', 'briosamarco', 'bru23', 'Cakes', 'CarlJ', 'chalant_af', 'chapinozo', 'chief10', 'chive', 'DANSALAHAM', 'Danube-Records', 'darkatek', 'darrelduke', 'Davros', 'DeepGrooveDJ', 'delay.rider', 'dfiant', 'diagonal', 'Discounter', 'diskoboy', 'diskskin', 'djhaloeight', 'do.you.remember', 'Dracu.', 'drubber77', 'drumphil', 'dzol1', 'ebeaudio', 'EIVISSAINLEEDS', 'elvoo', 'ennael', 'eoindkelly', 'EricCm', 'fabiolotto', 'Fair-Wax', 'filip.klockar', 'fitzmouse', 'Flatpack_Traxx', 'Freddy75', 'frgi', 'funkshun'], 'want': ['4cp', 'aaronarding', 'admrwe', 'adogcalypsenow', 'Adrien_bou', 'AndreaMazzucato', 'andylef

KeyError: 'sellers'

In [19]:
insert_release_sellers(p, releases)

KeyError: 'sellers'

In [22]:
releases[0]['sellers']

[{'image_url': None,
  'rating': 4.38,
  'have': 266,
  'want': 479,
  'title': 'Casy Hogan* - Uncut (12")',
  'label': 'Intrinsic Design',
  'catno': 'ID001-6',
  'media_condition': 'Generic',
  'media_condition_description': None,
  'seller': 'Recycle-Vinyl',
  'seller_rating': 4.5,
  'ships_from': 'United Kingdom',
  'currency': '£',
  'price': 16.99},
 {'image_url': None,
  'rating': 4.38,
  'have': 266,
  'want': 479,
  'title': 'Casy Hogan* - Uncut (12")',
  'label': 'Intrinsic Design',
  'catno': 'ID001-6',
  'media_condition': 'Generic',
  'media_condition_description': None,
  'seller': 'scottproblems',
  'seller_rating': 5.0,
  'ships_from': 'United Kingdom',
  'currency': '£',
  'price': 7.5},
 {'image_url': None,
  'rating': 4.38,
  'have': 266,
  'want': 479,
  'title': 'Casy Hogan* - Uncut (12")',
  'label': 'Intrinsic Design',
  'catno': 'ID001-6',
  'media_condition': 'Generic',
  'media_condition_description': None,
  'seller': 'ideefixe.nyc',
  'seller_rating': 4.5,
 

In [20]:
sellers_data = [
        (
            release['release_id'],
            seller['image_url'],
            seller['rating'],
            seller['have'],
            seller['want'],
            seller['title'],
            seller['label'],
            seller['catno'],
            seller['media_condition'],
            seller['media_condition_description'],
            seller['seller'],
            seller['seller_rating'],
            seller['ships_from'],
            seller['currency'],
            seller['price']
        )
        for release in releases for seller in release['sellers']
    ]

KeyError: 'sellers'

In [34]:
sellers = releases[0]['sellers']

In [37]:

for release in releases:
    if release.get('sellers'):
        print(release)

{'release_id': 17, 'release': {'Have': 271, 'Want': 497, 'Avg Rating': 4.38, 'Ratings': 55, 'Last Sold': datetime.date(2023, 12, 17), 'Low': 6.44, 'Median': 16.01, 'High': 21.51}, 'stats': {'have': ['andreeeeaaaaas', 'andygugs', 'asaisthorpe', 'Awesomer', 'bat_howell', 'bjoernmuenchau', 'blancodisco', 'bluediet', 'bobbsq', 'Bonna', 'briosamarco', 'bru23', 'Cakes', 'CarlJ', 'chalant_af', 'chapinozo', 'chief10', 'chive', 'DANSALAHAM', 'Danube-Records', 'darkatek', 'darrelduke', 'Davros', 'DeepGrooveDJ', 'delay.rider', 'dfiant', 'diagonal', 'Discounter', 'diskoboy', 'diskskin', 'djhaloeight', 'do.you.remember', 'Dracu.', 'drubber77', 'drumphil', 'dzol1', 'ebeaudio', 'EIVISSAINLEEDS', 'elvoo', 'ennael', 'eoindkelly', 'EricCm', 'fabiolotto', 'Fair-Wax', 'filip.klockar', 'fitzmouse', 'Flatpack_Traxx', 'Freddy75', 'frgi', 'funkshun'], 'want': ['4cp', 'aaronarding', 'admrwe', 'adogcalypsenow', 'Adrien_bou', 'AndreaMazzucato', 'andylef', 'andylfc4', 'Anzu16', 'Arnouth', 'ashwellb', 'Aurele', 'B